# Webscrap project: extracción de ofertas laborales de empleospúblico.cl en base a palabras clave

## Read Me
Este código utiliza técnicas de web scrapping y procesamiento de texto (utilizando regular expressions) para acceder a cada una de las más de 500 ofertas de empleos en el sitio web "empleospublicos.cl". El código identifica las ofertas laborales, extrae el texto descriptivo del perfil del cargo y de las especificaciones de la oferta laboral y finalmente genera una base de datos con aquellas ofertas laborales que cumplen con contener ciertas palabras clave que sugieran que es un cargo adecuado para mi perfil profesional.

Este código es 100\% de mi autoría como también la idea original.

## I) Extraer código HTML

El sitio web empleospublicos.cl contiene código dinámico, por lo que utilizamos Selenium para extraer manualmente el código.

In [13]:
import os

chromedriver_directorio = "C:\\Users\\rafae\\Downloads\\chromedriver-win64"
data_directorio = "C:\\Users\\rafae\\Documents\\github\\proyectos\\webscrap_ep_1\\data_extraida"

os.chdir(data_directorio)
os.getcwd()

'C:\\Users\\rafae\\Documents\\github\\proyectos\\webscrap_ep_1\\data_extraida'

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

#1) Utilizamos el paquete Selenium para ingresar a la pagina web y extraer el código html. Utilizamos Selenium puesto que el código de la página es dinámico.
s = Service(f"{chromedriver_directorio}\\chromedriver.exe") # Recordar descargar el driver compatible con la versión de google chrome
driver = webdriver.Chrome(service=s)

# Este código extrae información del sitio web empleospúblicos.cl
driver.get("https://www.empleospublicos.cl/") 

time.sleep(1) # agregamos tiempo de descanso al código para esperar que la página cargue correctamente
html_content = driver.page_source # extraemos código HTML
print("número de carácteres del código html: ",len(html_content))

driver.quit() # Cerramos la ventana de google chrome luego de extraer el código



#2) Utilizamos Beautifulsoup para poder trabajar más fácilmente con el código HTML
soup = BeautifulSoup(html_content, "lxml")

# A continuación generaremos una lista llamada items_href con todos los links de ofertas laborales en el sitio web
items = soup.find_all('div', class_='item') 
print("número de ofertas en empleospúblicos.cl:",len(items))
h3_href= items[1].find('h3').find('a', {'target': '_blank'}, href=True).get("href")
items_href=[]
for item in items:
    h3_href= item.find('h3').find('a', {'target': '_blank'}, href=True).get("href")
    items_href.append(h3_href)


    


número de carácteres del código html:  1887761
número de ofertas en empleospúblicos.cl: 526


## II) Iterar a través de las ofertas laborales

Luego de extraer el código html y los links de cada oferta laboral, obtenemos el código html de cada oferta para obtener todo el texto de la oferta laboral e identificar palabras claves.

In [17]:
import re

# Utilizamos un header que simule el ingreso de un usuario humano, esto sirve para evitar que el sitio web empleospublicos.cl impida acceder 
# a alguna url debido a algún tipo de bloqueo contra automatizaciones de extracción de datos.


#1) definimos parametros de la busqueda
headers = {
    'User-Agent': 'Your-App-Name/0.1',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer your_access_token'
}

#Generamos listas que contendrán la información de cada empleo relevante
economia_link=[]
economia_cargo=[]
economia_fecha=[]
economia_nuevo=[]
palabra_encontrada=[]

# Seleccionamos las palabras claves que el código buscará dentro de cada oferta laboral
words_to_find = ["ingeniería comercial", "ingeniero comercial", "economista", "ingeniero(a) comercial", "ingeniera comercial", "ingeniero/a comercial","ingeniero(a) comercial" ,"civil industrial", "analista", "ciencia de datos", "cientista de datos", "Python", "comercial" ]
#words_to_find = ["stata", "python", "sql"]


# la siguiente porción de código genera un dataframe nuevo en caso de no existir un archivo csv con ofertas laborales anteriores. La idea de esto es poder ir agregando 
# nuevas ofertas a archivos csv creados anteriormente con ofertas laborales antiguas.
try:
    df = pd.read_csv('data.csv')  
except (FileNotFoundError, NameError):
    # Si el archivo data.csv no existe, creamos nuevo dataframe
    data = {
    "Nombre Oferta": economia_cargo,
    "Plazo hasta": economia_fecha,
    "Link": economia_link,
    "Nuevo": economia_nuevo,
    "Palabra Clave": palabra_encontrada
}
    df = pd.DataFrame(data)  
link_set = set(df['Link'])



# creamos clase en caso de tener que lidiar con la ausencia de elementos html específicos, esto ocurre pues a medida que pasa
# el tiempo las clases e "id"s del código html van cambiando para algunas ofertas.
class ElementWithAttribute:
  def __init__(self, value):
    self.string = value



# A continuación utilizamos un loop que trabaja para identificar las palabras claves en cada foerta laboral
# por lo tanto el loop itera a través de la lista items_href que contiene cada link asociado a una oferta laboral
count=0
for item in items_href:
    count=count+1
    print("link número",count)
    
    # 1) conectarnos al link para extraer el código html
    try: 
        while True:     
            r = requests.get(item, headers=headers)
            if r.status_code == 200:
                break  # Exit the loop after successful response
    except:
        # En caso de no poder acceder al link, creamos una fila en el dataframe ( utilizando listas) que indique "error2"
        economia_link.append("error2")
        economia_cargo.append("error2")
        economia_fecha.append("error2")
        economia_nuevo.append("error2")
        palabra_encontrada.append("error2")
        continue
        
        
        
    # 2) trabajamos código html con beautiful soup        
    soup = BeautifulSoup(r.text, "lxml")
    soup_text=soup.get_text(strip=True).lower()
    
    # Ahora utilizamos expresiones regulares para ver si en el perfil del cargo están presentes las palabras claves definidas en la lista
    # "words_to_find" definida anteriormente
    for word in words_to_find:    
        if word in soup_text: 
            # Si la palabra clave se encuentra en el link, extraer la siguiente información y agregarla como fila al dataframe (mediante listas)
            # 2.1) agregamos link
            economia_link.append(item)
            # 2.2) agregamos nombre de cargo, el cual siempre sigue la misma id en todas las ofertas
            element = soup.find("span", id="lblAvisoTrabajo")
            if element!=None:
                cargo = element.string
            # a veces hay ofertas donde el cargo tiene otra id asociada en el código html:
            else:
                element = soup.find("span", id="lblTitulo")
                if element!=None:
                    element = element.find("strong")
                else:
                    element = ElementWithAttribute("error")
                if element==None:
                    element = ElementWithAttribute("error")
                cargo = element.string
            economia_cargo.append(cargo)
            print(cargo)
            
            # 2.3) agregamos fecha de cierre de postulación y la palabra clave encontrada
            element = soup.find("span", id="lblCalendarizacion")
            if element!=None:
                element2 = element.find_all("tr")
                element3 = element2[2].find_all("td")
                element3=element3[1]
                fechapostular = element3.string
                fechapostular = fechapostular.split("-")[1]
                economia_fecha.append(fechapostular)
                palabra_encontrada.append(word)
            else:
                element = soup.find("table", id="lblCalendarizacion")
                if element!=None:
                    element2 = element.find_all("tr")
                    element3 = element2[2].find_all("td")[-1]
                    fechapostular = element3.string
                    fechapostular = fechapostular.split("-")[1]
                    economia_fecha.append(fechapostular)
                    palabra_encontrada.append(word)
                else:
                    element2 = ElementWithAttribute("error")
                    element3 = ElementWithAttribute("error")
                    fechapostular = "error"
                    economia_fecha.append(fechapostular)
                    palabra_encontrada.append(fechapostular)
                    
            # Finalmente especificamos si la oferta encontrada estaba presente anteriormente en el dataframe o si es una oferta nueva  
            if item in link_set:
                economia_nuevo.append("-")
            else:
                economia_nuevo.append("nuevo")
            print("Oferta para economista en:", item)
            print(fechapostular)
            break # pasar a la siguiente iteración en caso de encontrar palabra clave



link número 1
link número 2
link número 3
link número 4
link número 5
Encargada (o) de Sección Gestión de Personas - Servicio de Biodiversidad y Áreas Protegidas - 121744
Oferta para economista en: https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=121744&c=0&j=0&tipo=convpostularavisoTrabajo
13/12/2024
link número 6
link número 7
link número 8
link número 9
link número 10
link número 11
link número 12
link número 13
link número 14
link número 15
link número 16
link número 17
link número 18
link número 19
link número 20
link número 21
link número 22
link número 23
link número 24
link número 25
link número 26
link número 27
link número 28
El Servicio Local de Educación Pública de Valparaíso busca Analista de Gestión de Proyectos e Infraestructura 13° EUS
Oferta para economista en: https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=115445&c=0&j=0&tipo=convpostularavisoTrabajo
12/12/2024
link número 29
link número 30
Analista Eco

## III) Crear dataframe con toda la información extríida

In [26]:
import pandas as pd
import datetime

# 1) Creamos archivo csv a partir de dataframe creado, con la información extraída de cada oferta laboral donde se encontró la presencia de una palabra clave
data = {
    "Nombre Oferta": economia_cargo,
    "Plazo hasta": economia_fecha,
    "Link": economia_link,
    "Nuevo": economia_nuevo,
    "Palabra Clave":  palabra_encontrada
}

df = pd.DataFrame(data)
df = df.sort_values(by='Palabra Clave')


now = datetime.datetime.now()
date_str = now.strftime("%Y-%m-%d")  
time_str = now.strftime("%H-%M-%S") 
filename = f"data_{date_str}_{time_str}.csv"  

print(filename)

df.to_csv(filename, mode='w', index=False, sep=';')
df

data_2024-12-12_20-09-22.csv


,Nombre Oferta,Plazo hasta,Link,Nuevo,Palabra Clave
27,Proceso de selección para proveer un cargo de ...,17/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
1,El Servicio Local de Educación Pública de Valp...,12/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
17,"ESTAMENTO PROFESIONAL, CALIDAD JURIDICA CONTRA...",18/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
23,Código 170: Médico Especialista – Servicio Neo...,22/01/2025,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
4,Analista Geoespacial – Unidad de Infraestructu...,13/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
5,"Técnico en Enfermería, 4to Turno para Unidad C...",18/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
32,Analista de Desarrollo TI - División de Gestió...,18/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
24,Código 168: Coordinador(a) Subunidad Clínica –...,12/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
28,Analista Ambiental y Territorial - Unidad de F...,16/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista
9,Código 171: Coordinador(a) Bronco - Infantil –...,18/12/2024,https://www.empleospublicos.cl/pub/convocatori...,nuevo,analista


In [23]:
for link in df["Link"]:
    print(link)

https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=122137&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=115445&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=120697&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=122310&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=120842&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=122336&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=122157&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=121843&c=0&j=0&tipo=convpostularaviso